In [1]:
import pandas as pd
import cv2
import pandasql

Rutas de input y output.

In [2]:
imp_img = '/Users/diegovillalizarraga/Maestria Ciencia de Datos ITAM/2do Semestre/Deep Learning/Proyecto Final/datos/Flicker8k_Dataset'
out_img = '/Users/diegovillalizarraga/Maestria Ciencia de Datos ITAM/2do Semestre/Deep Learning/Proyecto Final/text-to-image/102flowers'
out_txt = '/Users/diegovillalizarraga/Maestria Ciencia de Datos ITAM/2do Semestre/Deep Learning/Proyecto Final/text-to-image/text_c10/dog'

In [3]:
file = '/Users/diegovillalizarraga/Maestria Ciencia de Datos ITAM/2do Semestre/Deep Learning/Proyecto Final/datos/Flickr8k_text/Flickr8k.token.txt'
df_token = pd.read_csv(file, sep='\t', header=None)
df_token.columns = ['image', 'description']

In [4]:
df_token.head()

,image,description
0,1000268201_693b08cb0e.jpg#0,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg#1,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg#2,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg#3,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg#4,A little girl in a pink dress going into a woo...


In [5]:
df_token[['image', 'number']]= df_token.image.str.split("#",expand=True)

In [6]:
df_token = df_token.drop(['number'], axis=1)
df_token.head()

,image,description
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...


Filtramos perros.

In [7]:
temp = df_token.query('description.str.contains("dog")')

In [8]:
temp.head()

,image,description
5,1001773457_577c3a7d70.jpg,A black dog and a spotted dog are fighting
6,1001773457_577c3a7d70.jpg,A black dog and a tri-colored dog playing with...
7,1001773457_577c3a7d70.jpg,A black dog and a white dog with brown spots a...
8,1001773457_577c3a7d70.jpg,Two dogs of different breeds looking at each o...
9,1001773457_577c3a7d70.jpg,Two dogs on pavement moving toward each other .


In [9]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9274 entries, 5 to 40434
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   image        9274 non-null   object
 1   description  9274 non-null   object
dtypes: object(2)
memory usage: 217.4+ KB


In [10]:
sql = "SELECT DISTINCT image \
                FROM temp \
                ORDER BY image"
temp = pandasql.sqldf(sql, globals())
temp.head()

,image
0,1001773457_577c3a7d70.jpg
1,1003163366_44323f5815.jpg
2,1009434119_febe49276a.jpg
3,1012212859_01547e3f17.jpg
4,1015584366_dfcec3c85a.jpg


In [11]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2023 entries, 0 to 2022
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   image   2023 non-null   object
dtypes: object(1)
memory usage: 15.9+ KB


In [12]:
sql = "SELECT * \
            FROM df_token \
            WHERE image IN (SELECT DISTINCT image FROM temp) \
            ORDER BY image"
df_dog = pandasql.sqldf(sql, globals())
df_dog.head()

,image,description
0,1001773457_577c3a7d70.jpg,A black dog and a spotted dog are fighting
1,1001773457_577c3a7d70.jpg,A black dog and a tri-colored dog playing with...
2,1001773457_577c3a7d70.jpg,A black dog and a white dog with brown spots a...
3,1001773457_577c3a7d70.jpg,Two dogs of different breeds looking at each o...
4,1001773457_577c3a7d70.jpg,Two dogs on pavement moving toward each other .


Nos quedamos con los registros unicos de las imagenes de perros.

In [13]:
sql = "SELECT DISTINCT image \
                FROM df_dog \
                ORDER BY image"
img_unique = pandasql.sqldf(sql, globals())
img_unique.head()

,image
0,1001773457_577c3a7d70.jpg
1,1003163366_44323f5815.jpg
2,1009434119_febe49276a.jpg
3,1012212859_01547e3f17.jpg
4,1015584366_dfcec3c85a.jpg


In [14]:
img_unique.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2023 entries, 0 to 2022
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   image   2023 non-null   object
dtypes: object(1)
memory usage: 15.9+ KB


Agregamos hacemos un ranking que nos servira para el nuevo nombre de la imagen.

In [15]:
img_unique['ranking'] = img_unique['image'].rank(ascending=True)

In [16]:
img_unique.head(10)

,image,ranking
0,1001773457_577c3a7d70.jpg,1.0
1,1003163366_44323f5815.jpg,2.0
2,1009434119_febe49276a.jpg,3.0
3,1012212859_01547e3f17.jpg,4.0
4,1015584366_dfcec3c85a.jpg,5.0
5,101654506_8eb26cfb60.jpg,6.0
6,1019077836_6fc9b15408.jpg,7.0
7,1019604187_d087bf9a5f.jpg,8.0
8,1020651753_06077ec457.jpg,9.0
9,1022975728_75515238d8.jpg,10.0


In [17]:
img_unique.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2023 entries, 0 to 2022
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   image    2023 non-null   object 
 1   ranking  2023 non-null   float64
dtypes: float64(1), object(1)
memory usage: 31.7+ KB


In [18]:
len(img_unique['image'])

2023

Hacemos solo un caso antes de hacerlo función.

In [19]:
img = img_unique.iloc[0][0]
rank = int(img_unique.iloc[0][1])

In [20]:
img

'1001773457_577c3a7d70.jpg'

In [21]:
rank

1

Nos quedamos solo con las descripciones de esa imagen del dataframe de perros.

In [22]:
filtro = df_dog['image'] == img
desc_txt = df_dog[filtro]
desc_txt

,image,description
0,1001773457_577c3a7d70.jpg,A black dog and a spotted dog are fighting
1,1001773457_577c3a7d70.jpg,A black dog and a tri-colored dog playing with...
2,1001773457_577c3a7d70.jpg,A black dog and a white dog with brown spots a...
3,1001773457_577c3a7d70.jpg,Two dogs of different breeds looking at each o...
4,1001773457_577c3a7d70.jpg,Two dogs on pavement moving toward each other .


In [23]:
desc_txt = desc_txt.drop(['image'], axis=1)
desc_txt

,description
0,A black dog and a spotted dog are fighting
1,A black dog and a tri-colored dog playing with...
2,A black dog and a white dog with brown spots a...
3,Two dogs of different breeds looking at each o...
4,Two dogs on pavement moving toward each other .


Exportamos el txt a la carpeta de text_c10.

In [24]:
desc_txt.to_csv(f'{out_txt}/image_{rank}.txt', index=False, header=False)

Ya que vimos que funciona para un caso lo haremos para todos con un loop.

In [25]:
for i in range(len(img_unique['image'])):
    img = img_unique.iloc[i][0]
    rank = int(img_unique.iloc[i][1])
    filtro = df_dog['image'] == img
    desc_txt = df_dog[filtro]
    desc_txt = desc_txt.drop(['image'], axis=1)
    desc_txt.to_csv(f'{out_txt}/image_{rank}.txt', index=False, header=False)    

Ahora le cambiaremos el nombre a las imagenes.

Hacemos solo un caso antes de hacerlo función.

In [26]:
img = img_unique.iloc[0][0]
rank = int(img_unique.iloc[0][1])

In [27]:
img

'1001773457_577c3a7d70.jpg'

In [28]:
rank

1

Importamos la imagen.

In [29]:
file = cv2.imread(f'{imp_img}/{img}')

Exportamos la imagen.

In [30]:
cv2.imwrite(f'{out_img}/image_{rank}.jpg', file)

True

Ahora lo haremos para todos con un loop.

In [31]:
for i in range(len(img_unique['image'])):
    img = img_unique.iloc[i][0]
    rank = int(img_unique.iloc[i][1])
    file = cv2.imread(f'{imp_img}/{img}')
    cv2.imwrite(f'{out_img}/image_{rank}.jpg', file)